In [66]:
import requests
import json

requests.__version__

API_URL = 'http://localhost:8000/api/'
BASE_URL = 'http://localhost:8000/'
test = requests.get(API_URL + 'items')

In [53]:
json.loads(bytes.fromhex(test.content.hex()))

[{'name': 'Updated PUT Name', 'price': 1200.0},
 {'name': 'Baz', 'price': 12.34},
 {'name': 'Qux', 'price': 56.78},
 {'name': 'Quux', 'price': 45.67},
 {'name': 'Corge', 'price': 78.9},
 {'name': 'Grault', 'price': 90.12},
 {'name': 'Garply', 'price': 34.56},
 {'name': 'Waldo', 'price': 89.01},
 {'name': 'Fred', 'price': 67.23},
 {'name': 'Plugh', 'price': 45.89},
 {'name': 'Xyzzy', 'price': 23.78},
 {'name': 'Thud', 'price': 90.23},
 {'name': 'Another item', 'price': 44.0},
 {'name': 'Another item', 'price': 44.0},
 {'name': 'Another item', 'price': 44.0},
 {'name': 'Another item', 'price': 44.0},
 {'name': 'some item', 'price': 2.0},
 {'name': 'More item', 'price': '300'}]

##### Response GET params

In [59]:
def log_url(response, *args, **kwargs):
    print(f'Request URL: {response.url}')
      
query_params = {
    'offset': 2,
    'limit': 2,
    'max-price': 40,
}

response = requests.get(
    API_URL + 'items',
    params=query_params,
    hooks={'response': log_url}
)

print(response.json())

Request URL: http://localhost:8000/api/items?offset=2&limit=2&max-price=40
[{'name': 'Qux', 'price': 56.78}, {'name': 'Quux', 'price': 45.67}]


##### ERROR handling for accounting

In [1]:

import traceback

try:
    response = requests.get('http://127.0.0.1:8000/something')
    response.raise_for_status()
except Exception as http_err:
    print('--------------------------------------------########-----------------------------------------')
    print(f'HTTP error occured: {http_err}')
    print(f'INFO | {traceback.format_exc()}')
    print('--------------------------------------------########-----------------------------------------')
except Exception as err:
    print(err)
else:
    print(response.status_code)

--------------------------------------------########-----------------------------------------
HTTP error occured: name 'requests' is not defined
INFO | Traceback (most recent call last):
  File "/var/folders/bz/3q5l8rk53s989brg5n3nlrhc0000gn/T/ipykernel_61028/3627747889.py", line 4, in <module>
    response = requests.get('http://127.0.0.1:8000/something')
               ^^^^^^^^
NameError: name 'requests' is not defined

--------------------------------------------########-----------------------------------------


#### POST

##### JSON Requests

In [60]:
response = requests.post(
    API_URL + 'items',
    json={'name': 'some item', 'price': 2}
    #data={'name': 'Another item', 'price': 44},
    #allow_redirects=False
)

print(response.request.headers['content-type'])
print(response.request.body)

application/json
b'{"name": "some item", "price": 2}'


##### XML Requests

In [61]:
import xml.etree.ElementTree as ET
message_body = '''
    <item>
        <name>More item</name>
        <price>300</price>
    </item>
'''

response = requests.post(
    API_URL + 'items/xml',
    data=message_body,
    headers={'Content-Type': 'application/xml'}
)


print(ET.fromstring(response.text).find('name').text)
print(ET.fromstring(response.text).find('price').text)


More item
300


##### File Upload

In [71]:
file1 = open('./file.csv', 'rb')

files = [
    ('files', ('file.csv', file1, 'text/csv'))
]

response = requests.post(
    BASE_URL + 'upload-files',
    files=files
)

file1.close()
print(response.json())

{'uploaded_files': ['file.csv']}


##### MORE Requests

In [67]:
response = requests.put(
    
    API_URL + 'items/1',
    json={'name':'Updated PUT Name', 'price': 200}
)
print(response.json())

response = requests.patch( API_URL + 'items/1',
    json={'price': 1200}
)

response = requests.delete(API_URL + 'items/0')

{'name': 'Updated PUT Name', 'price': 200.0}


##### COOKIES

In [81]:
session = requests.Session()

with requests.Session() as session:
    credentials = {'username': 'some_name', 'password': 'pass'}

    login_response = session.post(API_URL + 'login', data=credentials)

    login_cookies = login_response.cookies

    print('Cookies returned from login:')
    print(session.cookies.get_dict())
    print('login response:')
    print(login_response.text)

    response = session.get(BASE_URL + 'protected', cookies=login_cookies)

    print('Protected route:')
    print(response.status_code)
    print(response.text)



Cookies returned from login:
{'user_id': '468c81512c1c58cf3478c006381c37fb'}
login response:
{"message":"Login successful"}
Protected route:
200
{"message":"You have access to this protected route"}


In [98]:
import logging
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import RetryError
from urllib3.util.retry import Retry

logging.basicConfig(level=logging.DEBUG)
requests_log = logging.getLogger('urllib3')
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

session = requests.Session()
retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[500], allowed_methods={'GET'})

session.mount('http:/127.0.0.1', HTTPAdapter(max_retries=retries))

try:
    response = session.get(BASE_URL + 'flaky')
    print(f'Final response status: {response.status_code}')

except RetryError:
    print('Maximum retries exceeded.Server is not available.')

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000
DEBUG:urllib3.connectionpool:http://localhost:8000 "GET /flaky HTTP/1.1" 200 21


Final response status: 200


##### Redirection

In [99]:
response = requests.head('http://127.0.0.1:8000/old-route', allow_redirects=True)

print(response.url)
print(response.status_code)
print(response.text)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "HEAD /old-route HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "HEAD /new-route HTTP/1.1" 200 0


http://127.0.0.1:8000/new-route
200



##### Basic authenitcation

In [100]:
import requests
from requests.auth import HTTPBasicAuth

username = 'username'
password = 'pass'

response = requests.get(
    'http://127.0.0.1:8000/protected-endpoint',
    auth=HTTPBasicAuth(username, password)
)

print(response.text)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "GET /protected-endpoint HTTP/1.1" 200 42


{"message":"Welcome, authenticated user!"}


In [101]:
pip install requests-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 2.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


#### Custom Authetnication method

In [102]:
import requests
from requests.auth import AuthBase

class JWTAuth(AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, requests):
        requests.headers['Authorization'] = f'Bearer {self.token}'
        
        return requests
    
token = 'abcde123'
response = requests.get(
    'http://127.0.0.1:8000/jwt-protected-route',
    auth=JWTAuth(token)
)

print(response.text)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "GET /jwt-protected-route HTTP/1.1" 200 47


{"message":"Access to protected route granted"}


import 